# โมเดลที่มีการแก้ไขสถาปัตยกรรม (เช่น เพิ่ม DSPPF, C2fEMA, EMA) ไม่สามารถโหลดได้จากไฟล์ .pt เพียงอย่างเดียว ต้องมีไฟล์ที่ประกาศคลาสของโมดูลที่ถูกปรับแต่ง เพื่อให้ runtime สามารถแปลงจาก best.pt >> best.onnx ได้ถูกต้อง

# 1) IMPORT CUSTOM MODULES
ต้องมีการแก้ไข line 5: path ไปยังตัวของ project_root

*   sys.path.append("D:/DCAS_project")   # Windows
*   sys.path.append("/home/user/DCAS_project")  # Linux

line 7 คือ module ที่มีการเพิ่มเข้าไปใน custom yolo ที่ต้องทำการ import

In [1]:
# =====================================================
# 1) ADD PROJECT ROOT TO PYTHON PATH
# =====================================================
import sys
sys.path.insert(0, "/content/DCAS_EMAdp")

# 2) Register Custom Layers ให้ระบบรู้จัก

Custom module ของ DCAS_EMAdp มี

*   DSPPF
*   C2fEMAdp
*   EMAdp

จึงต้องทำการ import ออกมา



In [2]:
# =====================================================
# 2) IMPORT CUSTOM MODULES FROM ultralytics.nn
# =====================================================
from ultralytics.nn.modules import DSPPF, C2fEMAdp, EMAdp

# 3) Register custom layers ให้ runtime รู้จัก

In [3]:
# =====================================================
# 3) REGISTER CUSTOM LAYERS INTO ULTRALYTICS
# =====================================================
import ultralytics.nn.modules as um

um.DSPPF = DSPPF
um.C2fEMA = C2fEMAdp
um.EMA = EMAdp

# 4) โหลดโมเดลที่เทรนไว้
ต้องมีการแก้ไข path ไปยังของ train weight เช่น C:/Users/Windows/DCAS_EMAdp/runs/train/gistda_point/yolov8+DCASEMAdp/weights/best.pt

*   model = YOLO("C:/Users/Windows/DCAS_EMAdp/runs/train/gistda_point/yolov8+DCASEMAdp/weights/best.pt ")   # Windows
*   model = YOLO("/home/user/DCAS_EMAdp/runs/train/gistda_point/yolov8+DCASEMAdp/weights/best.pt")  # Linux

In [4]:
# =====================================================
# 4) LOAD MODEL
# =====================================================
from ultralytics import YOLO
import torch
from ultralytics.nn.tasks import DetectionModel

torch.serialization.add_safe_globals([DetectionModel])

model = YOLO("/content/DCAS_EMAdp/runs/train/gistda_point/yolov8+DCASEMAdp/weights/best.pt")   # หรือ path เต็ม

/content/DCAS_EMAdp/ultralytics/nn/tasks.py:347: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


# 5) Export โมเดลเป็น ONNX

In [5]:
# =====================================================
# 5) EXPORT TO ONNX
# =====================================================
model.export(format="onnx")

print("EXPORT DONE")

Ultralytics YOLOv8.0.72 🚀 Python-3.12.12 torch-2.5.1+cu121 CPU
YOLOv8n+DCAS summary (fused): 202 layers, 1330579 parameters, 0 gradients, 11.7 GFLOPs

PyTorch: starting from /content/DCAS_EMAdp/runs/train/gistda_point/yolov8+DCASEMAdp/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 33600) (2.9 MB)

ONNX: starting export with onnx 1.20.1 opset 19...
ONNX: export success ✅ 1.5s, saved as /content/DCAS_EMAdp/runs/train/gistda_point/yolov8+DCASEMAdp/weights/best.onnx (5.8 MB)

Export complete (2.5s)
Results saved to /content/DCAS_EMAdp/runs/train/gistda_point/yolov8+DCASEMAdp/weights
Predict:         yolo predict task=detect model=/content/DCAS_EMAdp/runs/train/gistda_point/yolov8+DCASEMAdp/weights/best.onnx imgsz=640 
Validate:        yolo val task=detect model=/content/DCAS_EMAdp/runs/train/gistda_point/yolov8+DCASEMAdp/weights/best.onnx imgsz=640 data=/home/valentine/XDCAS/DCAS/ultralytics/datasets/spaceobject.yaml 
Visualize:       https://netron.app


EXPORT DONE


# ตัวอย่างการ นำ weight best.pt ของ DCAS_EMAdp ที่แปลงเป็น ONNX มาใช้งาน

In [6]:
import cv2
# Load the exported ONNX model
onnx_model = YOLO("/content/DCAS_EMAdp/runs/train/gistda_point/yolov8+DCASEMAdp/weights/best.onnx")

# Run inference
results = onnx_model("/content/2024_11_25_15_07_58_exp_5_000000_THAICOM8_flipV.png")
# save img
annotated_image = results[0].plot()
cv2.imwrite("DCAS_EMAdp_result.jpg", annotated_image)

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify', or 'pose'.
Loading /content/DCAS_EMAdp/runs/train/gistda_point/yolov8+DCASEMAdp/weights/best.onnx for ONNX Runtime inference...

image 1/1 /content/2024_11_25_15_07_58_exp_5_000000_THAICOM8_flipV.png: 640x640 2 objects, 270.9ms
Speed: 2.1ms preprocess, 270.9ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)


True